# P4Lang Tutorials of FABRIC

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for completing the P4 tutorials created by [P4Lang](https://github.com/p4lang/tutorials). The tutorials were origianlly designed to use a mininet topology. This example replaces the mininet topology with a FABRIC experiemnt topology that may span multiple sites across the FABRIC testbed.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [P4Lang Tutorials](https://github.com/p4lang/tutorials)
- [P4Lang YouTube Presentations](https://www.youtube.com/channel/UCOQAFkDKucJWr-KafdJsdIQ)

In [2]:
 import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='minawm_0041350787'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/Mina_Bastion_Key2' #'id_rsa_fabric'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Basic FABRIC Slice Configuration

In [3]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [4]:
try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
    available_resources.draw()
except Exception as e:
    print(f"Error: {e}")

Available Resources: Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
TACC        10  318/320  2552/2560  116390/116400  635/635                 2/2                             4/4                            16/16               4/4               6/6
UTAH        10  318/320  2552/2560  116390/116400  634/635                 2/2                             4/4                            16/16               4/4               5/5
WASH         6  188/192  1520/1536  60558/60600    380/381                 2/2                             2/2                            10/10               2/2               3/3
MICH         6  188/192  1520/1536  60580/60600    379/38

## Configure Slice Parameters

This section builds the experiment slice 

<img src="figs/fabric_slice.png" width="800"/>



In [5]:
# Slice 
slice_name = 'docker_tutorial_1'

node1_name = "n1"

switch_cores = 2
switch_ram = 8
switch_disk = 40

# Sites
site_1 = 'MAX'

image = 'default_ubuntu_20'
vlan = '1000'

### Create the Slice

In [8]:
try:
    #Create Slice
    slice1 = fablib.new_slice(name=slice_name)
    
    # Add switch node n1
    n1 = slice1.add_node(name=node1_name, site=site_1)
    n1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    n1.set_image(image)

    slice1.submit()
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    


-----------  ------------------------------------
Slice Name   docker_tutorial_1
Slice ID     ed19ad01-9da4-409a-b6bf-2840c6ea054f
Slice State  StableOK
Lease End    2022-04-02 13:32:39
-----------  ------------------------------------

Retry: 8, Time: 91 sec

ID                                    Name    Site    Host                         Cores    RAM    Disk  Image              Management IP    State    Error
------------------------------------  ------  ------  -------------------------  -------  -----  ------  -----------------  ---------------  -------  -------
ef5ecec3-29a3-462a-84b6-04e68856a23d  n1      MAX     max-w4.fabric-testbed.net        2      8     100  default_ubuntu_20  63.239.135.72    Active

Time to stable 91 seconds
Running wait_ssh ... Time to ssh 93 seconds
Running post_boot_config ... Time to post boot config 95 seconds


# timeout needs to be extended

## Get the Slice

In [10]:
try:
    slice1 = fablib.get_slice(name=slice_name)
    for node in slice1.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print(f"   Components        :  ")
        for component in node.get_components():
            print(f"      Name             : {component.get_name()}")
            print(f"      Details          : {component.get_details()}")
            print(f"      Disk (G)         : {component.get_disk()}")
            print(f"      Units            : {component.get_unit()}")
            print(f"      PCI Address      : {component.get_pci_addr()}")
            print(f"      Model            : {component.get_model()}")
            print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ")
        for interface in node.get_interfaces():
            print(f"       Name                : {interface.get_name()}")
            print(f"           Bandwidth           : {interface.get_bandwidth()}")
            print(f"           VLAN                : {interface.get_vlan()}")  
            print(f"           MAC                 : {interface.get_mac()}") 
            print(f"           OS iface name       : {interface.get_os_interface()}")
    for network in slice1.get_l2networks():
        print("Network:")
        print(f"    Name:            {network.get_name()}")
    print(f"Interface Map: {slice1.get_interface_map()}")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : n1
   Cores             : 2
   RAM               : 8
   Disk              : 100
   Image             : default_ubuntu_20
   Image Type        : qcow2
   Host              : max-w4.fabric-testbed.net
   Site              : MAX
   Management IP     : 63.239.135.72
   Reservation ID    : ef5ecec3-29a3-462a-84b6-04e68856a23d
   Reservation State : Active
   SSH Command       : ssh -i /home/fabric/.ssh/id_rsa -J minawm_0041350787@bastion-1.fabric-testbed.net ubuntu@63.239.135.72
   Components        :  
   Interfaces        :  
Interface Map: {}


In [11]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

node_ip_type = ""

try:
    slice1 = fablib.get_slice(name=slice_name)
    for node in slice1.get_nodes():
        if(node.get_name() == "n1"):
            if type(ip_address(node.get_management_ip())) is IPv4Address:
                node_ip_type = "IPv4"
            elif type(ip_address(node.get_management_ip())) is IPv6Address:
                node_ip_type = "IPv6"
            else:
                node_ip_type = "Unknown"
except Exception as e:
    print(f"Exception: {e}")
print(node_ip_type)

IPv4


## Node configuration.

In [13]:
n1 = slice1.get_node(name=node1_name) 

In [14]:
print(n1.execute("sudo apt-get update && sudo apt-get install -y docker.io")[0])

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1377 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [238 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [9876 B]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [853 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [122 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 c-n-f Metadata [532 B]
Get:12 http://security.ubuntu.com/ubuntu focal-security/univer

## If we pick a site that uses IPv4, we need to use the 'registry.ipv4.docker.com' prefix. If IPv6, we need to use the 'registry.ipv6.docker.com' prefix.

In [18]:
command = ""
if(node_ip_type == "IPv4"):
    command = "sudo docker run -d -it --cap-add=NET_ADMIN --privileged --name fabric_p4 registry.ipv4.docker.com/pruth/fabric-images:0.0.2j"
else:
    command = "sudo docker run -d -it --cap-add=NET_ADMIN --privileged --name fabric_p4 registry.ipv6.docker.com/pruth/fabric-images:0.0.2j"

In [19]:
print(n1.execute(command)[0])

162ffa1bc7f826d66a1e44a2590e34467036d784efd5d3457455fd63ba5c52f7



In [20]:
print(n1.execute("sudo docker container ls")[0])

CONTAINER ID   IMAGE                                                 COMMAND   CREATED         STATUS        PORTS     NAMES
162ffa1bc7f8   registry.ipv4.docker.com/pruth/fabric-images:0.0.2j   "bash"    6 seconds ago   Up 1 second             fabric_p4



## Delete Slice

In [21]:
try:
    slice1 = fablib.get_slice(name=slice_name)
    slice1.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()